# rpg_public_dronet Jupyter Notebook

## Introduction and Relevant Info

This notebook is a Jupyter conversion of this [repo](https://github.com/uzh-rpg/rpg_public_dronet).  You will need to clone the repo locally to your machine and copy this notebook into the `rpg_public_dronet` directory.

I go through the process of getting the needed data, preprocessing it, and training the model using keras and Tensorflow.

Before executing the code in this notebook, please note the following:

* This notebook was built using [Miniconda](https://conda.io/miniconda.html)
* The specific environment used can be built by using the `environment_gpu.yml` file included with the notebook.  For information on how to setup conda environments, see [this](https://conda.io/docs/user-guide/tasks/manage-environments.html).  You will need to choose the correct Tensorflow wheel for your operating system and gpu configuration; if you have a gpu, choose the `tensorflow_gpu` wheel, otherwise choose the cpu `tensorflow` wheel.

## Getting the Raw Data

* Download [the torrent files](https://github.com/udacity/self-driving-car/tree/master/datasets/CH2)
* Process/download using [Deluge](http://deluge-torrent.org/) or other

## Data Preprocessing

Data downloaded above are ROS bagfiles and need to be processed to get png images and steering info that the CNN requires.  This section goes through the conversion process.

### Bagfile Extraction

Python modules to import before beginning:

In [30]:
import subprocess
import os
import glob
import numpy as np
import re

To match the file structure in the repo, execute the following cells:

In [73]:
# USER CHANGE THESE TO FIT YOUR DATA
basedic = {'1':'/home/joe/ossdc/data/Ch2_002','2':'/home/joe/ossdc/data/HMB_3'}
tgtdir = '/home/joe/ossdc/rpg_data'

In [77]:
# Read bags
for d in basedic.keys():
    bags = [os.path.basename(x) for x in glob.glob(basedic[d] + "/*.bag")]
    for bag in bags:
        bagdir = re.sub(r'\.bag$', '', bag)
        print(bagdir)
        bashCommand = 'mkdir ' + tgtdir + '/' + bagdir
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
        bashCommand = 'cp -f ' + basedic[d] + '/' + bag + ' ' + tgtdir + '/' + bagdir + '/' + bag
        print(bashCommand)
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        #output, error = process.communicate()

HMB_3
cp -f /home/joe/ossdc/data/HMB_3/HMB_3.bag /home/joe/ossdc/rpg_data/HMB_3/HMB_3.bag
HMB_1
cp -f /home/joe/ossdc/data/Ch2_002/HMB_1.bag /home/joe/ossdc/rpg_data/HMB_1/HMB_1.bag
HMB_4
cp -f /home/joe/ossdc/data/Ch2_002/HMB_4.bag /home/joe/ossdc/rpg_data/HMB_4/HMB_4.bag
HMB_5
cp -f /home/joe/ossdc/data/Ch2_002/HMB_5.bag /home/joe/ossdc/rpg_data/HMB_5/HMB_5.bag
HMB_2
cp -f /home/joe/ossdc/data/Ch2_002/HMB_2.bag /home/joe/ossdc/rpg_data/HMB_2/HMB_2.bag
HMB_6
cp -f /home/joe/ossdc/data/Ch2_002/HMB_6.bag /home/joe/ossdc/rpg_data/HMB_6/HMB_6.bag


Now, the bags have been moved to a temporary directory for file extraction.  To extract the images, clone the following [repo](https://github.com/rwightman/udacity-driving-reader).

To use the shell scripts in the repo, you will need to download [docker-ce](https://docs.docker.com/install/linux/docker-ce/ubuntu/).  Follow the instructions included for bagfile extraction.  For your reference, the following shell commands should work (from the top-level of the repo):

```bash
./build.sh #builds the docker container
./run-bagdump.sh -i {root dir with folders containing bagfiles} -o {desired output dir} #processes the bagfiles

```

### Steps Outlined in rpg_public_dronet README.md

Two steps are described, which are:

* Change folder name
* Timestamp matching

#### Change folder name

In [79]:
basedir = '/home/joe/ossdc/rpg_public_dronet/data' #change to your data directory

In [82]:
for fn in os.listdir(basedir):
  if not os.path.isdir(os.path.join(basedir, fn)):
    continue # Not a directory
  else:
    print(fn)
    center = basedir + '/' + fn + '/center'
    images = basedir + '/' + fn + '/images'
    bashCommand = 'ln -sf ' + center + ' ' + images
    print(bashCommand)
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

HMB_5
ln -sf /home/joe/ossdc/rpg_public_dronet/data/HMB_5/center /home/joe/ossdc/rpg_public_dronet/data/HMB_5/images
HMB_3
ln -sf /home/joe/ossdc/rpg_public_dronet/data/HMB_3/center /home/joe/ossdc/rpg_public_dronet/data/HMB_3/images
HMB_6
ln -sf /home/joe/ossdc/rpg_public_dronet/data/HMB_6/center /home/joe/ossdc/rpg_public_dronet/data/HMB_6/images
HMB_4
ln -sf /home/joe/ossdc/rpg_public_dronet/data/HMB_4/center /home/joe/ossdc/rpg_public_dronet/data/HMB_4/images
HMB_1
ln -sf /home/joe/ossdc/rpg_public_dronet/data/HMB_1/center /home/joe/ossdc/rpg_public_dronet/data/HMB_1/images
HMB_2
ln -sf /home/joe/ossdc/rpg_public_dronet/data/HMB_2/center /home/joe/ossdc/rpg_public_dronet/data/HMB_2/images


#### Time Stamp Matching
The cells below implement [time_stamp_matching.py](https://github.com/uzh-rpg/rpg_public_dronet/blob/master/data_preprocessing/time_stamp_matching.py) from the repo.

In [83]:
def extractInfoFromFile(file_name):
    steer_stamps = []
    # Read file and extract time stamp
    try:
       steer_stamps = np.loadtxt(file_name, usecols=1, delimiter=',', skiprows=1, dtype=int)
    except:
        print(file_name)
    return steer_stamps


def getMatching(array1, array2):
    match_stamps = []
    match_idx = []
    for i in array1:
        dist = abs(i - array2)
        idx = np.where(dist == 0)[0]
        match_stamps.append(array2[idx])
        match_idx.append(idx)
    return match_stamps, match_idx


def getSyncSteering(fname, idx):
    mat = []
    try:
        mat = np.loadtxt(fname, usecols=(6,7,8,9,10,11), skiprows=1, delimiter=',')
        mat = mat[idx,:]
    except:
        print(fname)
    return mat

In [84]:
# Path to the data extracted from the Udacity dataset
folder = "./data" #"training"  or "testing"
assert folder, "You should provide the dataset folder"

In [85]:
experiments = glob.glob(folder + "/*")
# For every bag...
for exp in experiments:
    print(exp)
    # Read images
    images = [os.path.basename(x) for x in glob.glob(exp + "/images/*.png")]
    im_stamps = []
    for im in images:
        stamp = int(re.sub(r'\.png$', '', im))
        #print(stamp)
        im_stamps.append(stamp)
    im_stamps = np.array(sorted(im_stamps))

    # Extract time stamps from steerings
    file_name = exp + "/interpolated.csv"
    steer_stamps = extractInfoFromFile(file_name)

    # Time-stamp matching between images and steerings
    match_stamp, match_idx = getMatching(im_stamps, steer_stamps)
    match_idx = np.array(match_idx)
    print(match_idx.shape)
    try:
        match_idx = match_idx[:,0]
        # Get matched commands
        original_fname = exp + "/interpolated.csv"
        sync_steer = getSyncSteering(original_fname, match_idx)
        new_fname = exp + "/sync_steering.txt"
        np.savetxt(new_fname, sync_steer, delimiter=',',
                   header="angle,torque,speed,lat,long,alt")
    except:
        continue

./data/HMB_5
(4235, 1)
./data/HMB_3
(5279, 1)
./data/HMB_6
(7402, 1)
./data/HMB_4
(1974, 1)
./data/HMB_1
(4401, 1)
./data/HMB_2
(15796, 1)


### Creating the training, validation, and testing folders

In [112]:
basedir = './data'

bashCommand = 'mkdir -p ' + basedir + '/training'
print(bashCommand)
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

bashCommand = 'mkdir -p ' + basedir + '/validation'
print(bashCommand)
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

bashCommand = 'mkdir -p ' + basedir + '/testing'
print(bashCommand)
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

for exp in experiments:
    expname = re.sub(r'./data/', '', exp)
    #print(expname)
    if expname == 'HMB_1' or expname == 'HMB_2' or expname == 'HMB_5' or expname == 'HMB_6':
        bashCommand0 = 'mkdir -p ' + basedir + '/training/' + expname + '/images'
        bashCommand1 = 'cp -f ' + exp + '/center/*.png ' + basedir + '/training/' + expname + '/images'
        bashCommand2 = 'cp -f ' + exp + '/sync_steering.txt ' + basedir + '/training/' + expname + '/'
    elif exp == './data/HMB_4':
        bashCommand0 = 'mkdir -p ' + basedir + '/validation/' + expname + '/images'
        bashCommand1 = 'cp -f ' + exp + '/center/*.png ' + basedir + '/validation/' + expname + '/images'
        bashCommand2 = 'cp -f ' + exp + '/sync_steering.txt ' + basedir + '/validation/' + expname + '/'
    else:
        bashCommand0 = 'mkdir -p ' + basedir + '/testing/' + expname + '/images'
        bashCommand1 = 'cp -f ' + exp + '/center/*.png ' + basedir + '/testing/' + expname + '/images/'
        bashCommand2 = 'cp -f ' + exp + '/sync_steering.txt ' + basedir + '/testing/' + expname + '/'
    print(bashCommand0)
    print(bashCommand1)
    print(bashCommand2)
    process = subprocess.Popen(bashCommand0.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    process = subprocess.Popen(bashCommand1.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    print(error)
    process = subprocess.Popen(bashCommand2.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

mkdir -p ./data/training
mkdir -p ./data/validation
mkdir -p ./data/testing
mkdir -p ./data/training/HMB_5/images
cp -f ./data/HMB_5/center/*.png ./data/training/HMB_5/images
cp -f ./data/HMB_5/sync_steering.txt ./data/training/HMB_5/
None
mkdir -p ./data/testing/HMB_3/images
cp -f ./data/HMB_3/center/*.png ./data/testing/HMB_3/images/
cp -f ./data/HMB_3/sync_steering.txt ./data/testing/HMB_3/
None
mkdir -p ./data/training/HMB_6/images
cp -f ./data/HMB_6/center/*.png ./data/training/HMB_6/images
cp -f ./data/HMB_6/sync_steering.txt ./data/training/HMB_6/
None
mkdir -p ./data/validation/HMB_4/images
cp -f ./data/HMB_4/center/*.png ./data/validation/HMB_4/images
cp -f ./data/HMB_4/sync_steering.txt ./data/validation/HMB_4/
None
mkdir -p ./data/training/HMB_1/images
cp -f ./data/HMB_1/center/*.png ./data/training/HMB_1/images
cp -f ./data/HMB_1/sync_steering.txt ./data/training/HMB_1/
None
mkdir -p ./data/training/HMB_2/images
cp -f ./data/HMB_2/center/*.png ./data/training/HMB_2/images
c

### Final Cleanup

After verification of proper formatting, remove the `HMB_*` directories from the top-level data directory:

```bash
rm -rf {path_to_data}/HMB_*
```

## Training DroNet

In [1]:
# check if gpu support is available
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Output (to terminal running this notebook) was:

```bash

I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcublas.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcudnn.so.5 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcufft.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcurand.so.8.0 locally
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE3 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.
I tensorflow/core/common_runtime/gpu/gpu_device.cc:885] Found device 0 with properties: 
name: GeForce GTX 1080 Ti
major: 6 minor: 1 memoryClockRate (GHz) 1.582
pciBusID 0000:01:00.0
Total memory: 10.91GiB
Free memory: 10.14GiB
I tensorflow/core/common_runtime/gpu/gpu_device.cc:906] DMA: 0 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:916] 0:   Y 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:975] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0)
Device mapping:
/job:localhost/replica:0/task:0/gpu:0 -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0
I tensorflow/core/common_runtime/direct_session.cc:257] Device mapping:
/job:localhost/replica:0/task:0/gpu:0 -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0

```

Tensorflow found a GTX1080Ti and will use it.

Now, you should be ready to train the CNN.  _Note: I had issues with pydot and vizgraph when trying to export a png image of the CNN.  After commenting out the lines of cnn.py responsible for exporting the architecture.png file, I was able to train the network without further problems._

In [2]:
%run cnn.py --experiment_rootdir='./model/test_1' --train_dir='./data/training' --val_dir='./data/validation' --batch_size=16 --epochs=150 --log_rate=25

Using TensorFlow backend.
/home/joe/miniconda3/envs/rpg_public_dronet/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Found 61696 images belonging to 131 experiments.
Found 2508 images belonging to 3 experiments.
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200, 200, 1)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 100, 100, 32)  832                                          
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 49, 49, 32)    0                                            
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 49, 49, 32)    128                                       

/home/joe/miniconda3/envs/rpg_public_dronet/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/joe/miniconda3/envs/rpg_public_dronet/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:2124: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


1.0
0.0
Epoch 1/150
3855/3856 [============================>.] - ETA: 0s - loss: 0.1077 - dense_1_loss: 0.0461 - activation_8_loss: 0.3409-------------------------------------
|      train_loss |           0.108 |
|        val_loss |          0.0912 |
-------------------------------------
3856/3856 [==============================] - 1334s - loss: 0.1077 - dense_1_loss: 0.0461 - activation_8_loss: 0.3408 - val_loss: 0.0912 - val_dense_1_loss: 0.0452 - val_activation_8_loss: 0.1524
1.0
0.0
Epoch 2/150
3855/3856 [============================>.] - ETA: 0s - loss: 0.0622 - dense_1_loss: 0.0369 - activation_8_loss: 0.3353-------------------------------------
|      train_loss |          0.0622 |
|        val_loss |          0.0557 |
-------------------------------------
3856/3856 [==============================] - 1223s - loss: 0.0622 - dense_1_loss: 0.0369 - activation_8_loss: 0.3353 - val_loss: 0.0557 - val_dense_1_loss: 0.0476 - val_activation_8_loss: 0.1474
1.0
0.0
Epoch 3/150
3855/3856 

3856/3856 [==============================] - 675s - loss: 0.1359 - dense_1_loss: 0.0385 - activation_8_loss: 0.2159 - val_loss: 0.0737 - val_dense_1_loss: 0.0470 - val_activation_8_loss: 0.0592
1.0
0.5034147
Epoch 18/150
3855/3856 [============================>.] - ETA: 0s - loss: 0.1472 - dense_1_loss: 0.0385 - activation_8_loss: 0.2160-------------------------------------
|      train_loss |           0.147 |
|        val_loss |          0.0808 |
-------------------------------------
3856/3856 [==============================] - 652s - loss: 0.1472 - dense_1_loss: 0.0385 - activation_8_loss: 0.2159 - val_loss: 0.0808 - val_dense_1_loss: 0.0475 - val_activation_8_loss: 0.0663
1.0
0.55067104
Epoch 19/150
3855/3856 [============================>.] - ETA: 0s - loss: 0.1574 - dense_1_loss: 0.0385 - activation_8_loss: 0.2159-------------------------------------
|      train_loss |           0.157 |
|        val_loss |          0.0818 |
-------------------------------------
3856/3856 [======

...

3856/3856 [==============================] - 662s - loss: 0.5153 - dense_1_loss: 0.0560 - activation_8_loss: 0.4596 - val_loss: 0.2047 - val_dense_1_loss: 0.0661 - val_activation_8_loss: 0.1387
1.0
0.99932444
Epoch 84/150
3855/3856 [============================>.] - ETA: 0s - loss: 0.5155 - dense_1_loss: 0.0560 - activation_8_loss: 0.4598-------------------------------------
|      train_loss |           0.516 |
|        val_loss |            0.22 |
-------------------------------------
3856/3856 [==============================] - 651s - loss: 0.5156 - dense_1_loss: 0.0560 - activation_8_loss: 0.4599 - val_loss: 0.2201 - val_dense_1_loss: 0.0700 - val_activation_8_loss: 0.1503
1.0
0.99938875
Epoch 85/150
3855/3856 [============================>.] - ETA: 0s - loss: 0.5150 - dense_1_loss: 0.0560 - activation_8_loss: 0.4593-------------------------------------
|      train_loss |           0.515 |
|        val_loss |           0.232 |
-------------------------------------
3856/3856 [=====

3855/3856 [============================>.] - ETA: 0s - loss: 0.5641 - dense_1_loss: 0.0559 - activation_8_loss: 0.5083-------------------------------------
|      train_loss |           0.564 |
|        val_loss |           0.255 |
-------------------------------------
Saved model at ./model/test_1/model_weights_99.h5
3856/3856 [==============================] - 653s - loss: 0.5641 - dense_1_loss: 0.0560 - activation_8_loss: 0.5082 - val_loss: 0.2549 - val_dense_1_loss: 0.0701 - val_activation_8_loss: 0.1847
1.0
0.9998766
Epoch 101/150
3855/3856 [============================>.] - ETA: 0s - loss: 0.5640 - dense_1_loss: 0.0560 - activation_8_loss: 0.5081-------------------------------------
|      train_loss |           0.564 |
|        val_loss |           0.265 |
-------------------------------------
3856/3856 [==============================] - 652s - loss: 0.5640 - dense_1_loss: 0.0560 - activation_8_loss: 0.5081 - val_loss: 0.2645 - val_dense_1_loss: 0.0686 - val_activation_8_loss: 0

3856/3856 [==============================] - 664s - loss: 0.5695 - dense_1_loss: 0.0616 - activation_8_loss: 0.5079 - val_loss: 0.2595 - val_dense_1_loss: 0.0771 - val_activation_8_loss: 0.1824
1.0
0.9999751
Epoch 117/150
3855/3856 [============================>.] - ETA: 0s - loss: 0.5698 - dense_1_loss: 0.0616 - activation_8_loss: 0.5082-------------------------------------
|      train_loss |            0.57 |
|        val_loss |           0.273 |
-------------------------------------
3856/3856 [==============================] - 649s - loss: 0.5698 - dense_1_loss: 0.0616 - activation_8_loss: 0.5082 - val_loss: 0.2730 - val_dense_1_loss: 0.0752 - val_activation_8_loss: 0.1979
1.0
0.99997747
Epoch 118/150
3855/3856 [============================>.] - ETA: 0s - loss: 0.5698 - dense_1_loss: 0.0616 - activation_8_loss: 0.5082-------------------------------------
|      train_loss |            0.57 |
|        val_loss |           0.259 |
-------------------------------------
3856/3856 [====

3855/3856 [============================>.] - ETA: 0s - loss: 0.6244 - dense_1_loss: 0.0616 - activation_8_loss: 0.5628-------------------------------------
|      train_loss |           0.624 |
|        val_loss |           0.318 |
-------------------------------------
3856/3856 [==============================] - 653s - loss: 0.6244 - dense_1_loss: 0.0616 - activation_8_loss: 0.5628 - val_loss: 0.3175 - val_dense_1_loss: 0.0753 - val_activation_8_loss: 0.2422
1.0
0.99999547
Epoch 134/150
3855/3856 [============================>.] - ETA: 0s - loss: 0.6235 - dense_1_loss: 0.0616 - activation_8_loss: 0.5619-------------------------------------
|      train_loss |           0.624 |
|        val_loss |           0.327 |
-------------------------------------
3856/3856 [==============================] - 651s - loss: 0.6236 - dense_1_loss: 0.0616 - activation_8_loss: 0.5620 - val_loss: 0.3274 - val_dense_1_loss: 0.0771 - val_activation_8_loss: 0.2503
1.0
0.9999959
Epoch 135/150
3855/3856 [====

3856/3856 [==============================] - 675s - loss: 0.6240 - dense_1_loss: 0.0616 - activation_8_loss: 0.5624 - val_loss: 0.3264 - val_dense_1_loss: 0.0772 - val_activation_8_loss: 0.2492
1.0
0.9999991
Epoch 150/150
3855/3856 [============================>.] - ETA: 0s - loss: 0.6233 - dense_1_loss: 0.0616 - activation_8_loss: 0.5617-------------------------------------
|      train_loss |           0.623 |
|        val_loss |           0.318 |
-------------------------------------
Saved model at ./model/test_1/model_weights_149.h5
3856/3856 [==============================] - 649s - loss: 0.6232 - dense_1_loss: 0.0616 - activation_8_loss: 0.5616 - val_loss: 0.3181 - val_dense_1_loss: 0.0774 - val_activation_8_loss: 0.2407


## Evaluate the Model

In [4]:
%run evaluation.py --experiment_rootdir='./model/test_1' --weights_fname='model_weights_149.h5' --test_dir='./data/testing'

Found 6855 images belonging to 9 experiments.
Loaded model from ./model/test_1/model_weights_149.h5
429/429 [==============================] - 70s    
EVA = 0.0
RMSE = 0.21290822327136993
Written file ./model/test_1/constant_regression.json
EVA = 0.0
RMSE = 0.21460199356079102
Written file ./model/test_1/test_regression.json
EVA = -1.0482769993956822
RMSE = 0.30472281730667894
Written file ./model/test_1/random_regression.json
Written file ./model/test_1/predicted_and_real_steerings.json
Average accuracy =  0.5145939086294417
Precision =  0.22086720867208673
Recall =  0.22086720867208673
F1-score =  0.29880843263061413
Written file ./model/test_1/random_classification.json
Average accuracy =  0.7760152284263959
Precision =  0.0
Recall =  0.0
F1-score =  0.0
Written file ./model/test_1/test_classification.json
Written file ./model/test_1/predicted_and_real_labels.json


/home/joe/miniconda3/envs/rpg_public_dronet/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/joe/miniconda3/envs/rpg_public_dronet/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
